# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [47]:
from datetime import datetime

from pyspark.sql import SparkSession

from pyspark.sql.functions import min as smin, max as smax, sum as ssum, round as sround
from pyspark.sql.functions import isnan, isnull, when, first, avg, last, count, countDistinct, col, lag, lead, coalesce, lit, split, trim

from pyspark.sql.window import Window
from pyspark.sql.functions import to_date, date_format, from_unixtime, to_timestamp

from pyspark.sql.types import DateType, TimestampType, IntegerType
 
import jupyter_utils as j

from pyspark import SparkContext
 
SparkContext.setSystemProperty('spark.logConf', 'True')
SparkContext.setSystemProperty('spark.default.parallelism', '16')
SparkContext.setSystemProperty('spark.executor.memory', '6g')
SparkContext.setSystemProperty('spark.driver.memory', '4g')
SparkContext.setSystemProperty('spark.reducer.maxSizeInFlight', '96m')
SparkContext.setSystemProperty('spark.shuffle.consolidateFiles', 'True') 
SparkContext.setSystemProperty('spark.shuffle.service.index.cache.size', '500m')

In [2]:
j.reload(j)

In [21]:
# filepath = 'sparkify_full_csv_data.csv'
filepath = 'medium_sparkify_event_data.json'

In [51]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

spark.sparkContext.setLogLevel('INFO')

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.driver.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.shuffle.service.index.cache.size', '500m'),
 ('spark.executor.memory', '6g'),
 ('spark.reducer.maxSizeInFlight', '96m'),
 ('spark.shuffle.consolidateFiles', 'True'),
 ('spark.driver.port', '36915'),
 ('spark.default.parallelism', '16'),
 ('spark.logConf', 'True'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1584219847607'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.0.107')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [142]:
# df = spark.read.option("inferSchema", "true").option("header", "true").option("encoding", "utf-8").csv(filepath)
df = spark.read.option("inferSchema", "true").option("header", "true").option("encoding", "utf-8").json(filepath)

In [23]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [35]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|  song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+
|110828|   0|    15700| 15700|            0|   15700|110828|    0|   15700|     0|   0|       15700|        0|110828|     0|  0|    15700|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+



In [38]:
df.groupby('auth').agg(count(col('auth'))).show(50)

+----------+-----------+
|      auth|count(auth)|
+----------+-----------+
|Logged Out|      15606|
| Cancelled|         99|
|     Guest|         94|
| Logged In|     527906|
+----------+-----------+



In [46]:
df.where(~df.auth.isin(['Logged In', 'Cancelled'])).show(1)

+------+----------+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|          186|    null|  null| paid|    null|   GET|Home|        null|      178|null|   200|1538352148000|     null|      |
+------+----------+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+-------------+---------+------+
only showing top 1 row



In [43]:
df.where((df.auth == 'Logged In') & (df.page == 'Home')).show(10)

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----+-------------+---------+----+------+-------------+--------------------+------+
|  null|Logged In| Maverick|     M|          190|Santiago|  null| paid|Orlando-Kissimmee...|   GET|Home|1535953455000|      178|null|   200|1538352176000|"Mozilla/5.0 (Mac...|   179|
|  null|Logged In|    Colin|     M|            0|  Larson|  null| free|Dallas-Fort Worth...|   GET|Home|1537982255000|      497|null|   200|1538352241000|Mozilla/5.0 (Wind...|   100|
|  null|Logged In|  Gilbert|     M|            7|    Todd|  null| free|Charlotte-Conc

In [61]:
log4jLogger = spark.sparkContext._jvm.org.apache.log4j

LOGGER = log4jLogger.LogManager.getLogger('driver_logger')

def info(message, print_on_notebook = True):
    LOGGER.info(message)
    
    if print_on_notebook:
        print(message)
    
info('Logger instance created')

Logger instance created


In [143]:
def clean_dataframe(df):
    
    info('Starting data cleaning...')
    
    total_before = df.count()
    
    # Keep only logged records
    df = df.where(df.auth.isin(['Logged In', 'Cancelled']))
    
    # Records without userId
    df = df.where(col('userId').isNotNull())
    
    # Create a date column for the event
    df = df.withColumn('date', from_unixtime(col('ts') / 1000).cast(DateType()))
    
    # Location
    df = df.withColumn('state', trim(split((split('location', ',').getItem(1)), '-').getItem(0)))
    
    total_removed = total_before - df.count()
    
    info('Finished data cleaning...')
    info(f'Number of removed rows: {total_removed}')
    
    return df

In [144]:
df = clean_dataframe(df)

Starting data cleaning...
Finished data cleaning...
Number of removed rows: 15700


In [9]:
from pyspark import StorageLevel

In [10]:
print(df.storageLevel)

df.persist(StorageLevel.MEMORY_ONLY)

print(df.storageLevel)

Serialized 1x Replicated
Memory Serialized 1x Replicated


In [11]:
df.storageLevel

StorageLevel(False, True, False, False, 1)

In [16]:
df.dtypes

[('gender', 'string'),
 ('length', 'double'),
 ('level', 'string'),
 ('registration', 'double'),
 ('userId', 'int'),
 ('ts', 'bigint'),
 ('page', 'string'),
 ('sessionId', 'int'),
 ('itemInSession', 'int')]

In [13]:
df.show(n=2, truncate=False, vertical=True)

-RECORD 0-----------------------
 gender        | F              
 length        | null           
 level         | paid           
 registration  | 1.5347687E12   
 userId        | 1229546        
 ts            | 1542110625000  
 page          | Add Friend     
 sessionId     | 184768         
 itemInSession | 54             
-RECORD 1-----------------------
 gender        | M              
 length        | null           
 level         | free           
 registration  | 1.533842212E12 
 userId        | 1159643        
 ts            | 1542110625000  
 page          | Home           
 sessionId     | 180496         
 itemInSession | 39             
only showing top 2 rows



In [29]:
df.where(df.userId == 1370646).groupby('page').agg(count)

749

In [39]:
df_days.show(n = 1)

+-------+------+
| userId|n_days|
+-------+------+
|1321436|    20|
+-------+------+



In [43]:
df_daily_actions.show(n = 1)

+-------+-----------------+
| userId|avg_daily_actions|
+-------+-----------------+
|1321436|            30.75|
+-------+-----------------+



In [46]:
df_days.join(df_daily_actions, df_days.userId == df_daily_actions.userId).drop(df_daily_actions.userId).show(1)

+------+-------+-----------------+
|n_days| userId|avg_daily_actions|
+------+-------+-----------------+
|    20|1321436|            30.75|
+------+-------+-----------------+



In [27]:
from pyspark.sql.functions import from_unixtime, to_timestamp
from pyspark.sql.types import DateType

df.withColumn('date', from_unixtime(col('ts') / 1000).cast(DateType())).groupby('userId').agg(countDistinct('date')).show(n = 50)

+-------+--------------------+
| userId|count(DISTINCT date)|
+-------+--------------------+
|1321436|                  20|
|1454596|                  38|
|1376661|                  10|
|1883770|                  36|
|1437805|                   9|
|1127759|                   9|
|1997884|                  23|
|1588738|                  17|
|1920861|                  27|
|1034728|                  12|
|1862793|                  13|
|1957152|                  17|
|1763951|                  21|
|1933042|                  30|
|1699511|                  19|
|1760700|                   9|
|1760879|                  27|
|1117946|                  26|
|1629144|                  17|
|1860076|                  21|
|1711766|                  41|
|1721808|                  49|
|1178536|                  18|
|1786670|                  10|
|1702129|                  34|
|1226805|                  27|
|1940021|                  13|
|1392770|                  28|
|1776107|                  12|
|1501768

In [7]:
df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [50]:
not_na_columns = [ 'userId', 'sessionId' ]

In [15]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|     0|   0|        0|     0|            0|       0|     0|    0|       0|     0|   0|           0|        0|   0|     0|  0|        0|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+



In [16]:
df.groupBy('userId').count().orderBy('count', ascending = False).show(50)

+------+-----+
|userId|count|
+------+-----+
|      |15700|
|    92| 9767|
|   140| 7448|
|300049| 7309|
|   101| 6842|
|300035| 6810|
|   195| 6184|
|   230| 6019|
|   163| 5965|
|   250| 5678|
|    18| 5511|
|   276| 5346|
|300017| 5266|
|    87| 5243|
|   293| 5125|
|300021| 5076|
|    42| 4952|
|300011| 4816|
|    30| 4737|
|    12| 4232|
|300031| 4194|
|   126| 4190|
|   283| 4181|
|   228| 4092|
|   100| 3999|
|   259| 3633|
|   105| 3597|
|   246| 3566|
|   121| 3541|
|   269| 3511|
|   292| 3504|
|    70| 3465|
|    35| 3456|
|    38| 3211|
|    98| 3206|
|   282| 3191|
|   185| 3088|
|300023| 3018|
|100009| 2987|
|   157| 2966|
|200023| 2955|
|   104| 2950|
|   174| 2917|
|   225| 2849|
|300038| 2829|
|   172| 2728|
|    85| 2696|
|   258| 2684|
|200020| 2654|
|   179| 2639|
+------+-----+
only showing top 50 rows



In [17]:
df.select("artist").distinct().count()

21248

In [18]:
df.select('length').describe().show()

+-------+------------------+
|summary|            length|
+-------+------------------+
|  count|            432877|
|   mean|248.66459278007508|
| stddev| 98.41266955052019|
|    min|           0.78322|
|    max|        3024.66567|
+-------+------------------+



In [13]:
print(f'Rows before: {df.count()}')

df = df.where(df.userId != '')

print(f'Rows after: {df.count()}')

Rows before: 26259199
Rows after: 0


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [20]:
df.groupBy('page').count().orderBy('count', ascending = False).show(50)

+--------------------+------+
|                page| count|
+--------------------+------+
|            NextSong|432877|
|           Thumbs Up| 23826|
|                Home| 19089|
|     Add to Playlist| 12349|
|          Add Friend|  8087|
|         Roll Advert|  7773|
|              Logout|  5990|
|         Thumbs Down|  4911|
|           Downgrade|  3811|
|            Settings|  2964|
|                Help|  2644|
|               About|  1026|
|             Upgrade|   968|
|       Save Settings|   585|
|               Error|   503|
|      Submit Upgrade|   287|
|    Submit Downgrade|   117|
|              Cancel|    99|
|Cancellation Conf...|    99|
+--------------------+------+



Some questions about the data:

- Are errors related to downgrading canceling the service?
- Having a certain number of friends or a sense of community can decrease the churn?
- Thumbs down are related to churn? (could the quality of the songs catalog affect the churn)
- The advertising is not annoying the users?
- Users with stay connected for more time have less change to churn?
- Is the home page relevant?
- Users, who access the downgrade page are how much more willing to churn?

In [21]:
df.groupBy('status').count().orderBy('count', ascending = False).show(20)

+------+------+
|status| count|
+------+------+
|   200|483600|
|   307| 43902|
|   404|   503|
+------+------+



In [15]:
df.filter('userId = 92').groupBy('page').count().orderBy('count', ascending = False).show(50)

+----------------+-----+
|            page|count|
+----------------+-----+
|        NextSong| 8177|
|       Thumbs Up|  400|
|            Home|  308|
| Add to Playlist|  248|
|      Add Friend|  158|
|          Logout|   96|
|       Downgrade|   85|
|     Thumbs Down|   80|
|            Help|   62|
|     Roll Advert|   60|
|        Settings|   48|
|           About|   17|
|           Error|   13|
|         Upgrade|    7|
|   Save Settings|    5|
|  Submit Upgrade|    2|
|Submit Downgrade|    1|
+----------------+-----+



In [22]:
df.filter('userId = 92').groupBy('page').count().orderBy('count', ascending = False).show(50)

+----------------+-----+
|            page|count|
+----------------+-----+
|        NextSong| 8177|
|       Thumbs Up|  400|
|            Home|  308|
| Add to Playlist|  248|
|      Add Friend|  158|
|          Logout|   96|
|       Downgrade|   85|
|     Thumbs Down|   80|
|            Help|   62|
|     Roll Advert|   60|
|        Settings|   48|
|           About|   17|
|           Error|   13|
|         Upgrade|    7|
|   Save Settings|    5|
|  Submit Upgrade|    2|
|Submit Downgrade|    1|
+----------------+-----+



In [24]:
df.filter('userId = 92').groupBy('userAgent').count().orderBy('count', ascending = False).show(50, False)

+----------------------------------------------------------------------------------------------------------------------------------+-----+
|userAgent                                                                                                                         |count|
+----------------------------------------------------------------------------------------------------------------------------------+-----+
|"Mozilla/5.0 (iPad; CPU OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"|9767 |
+----------------------------------------------------------------------------------------------------------------------------------+-----+



In [25]:
df.filter('userId = 92 and song != \'null\' ').groupBy('song').count().orderBy('count', ascending = False).show(50, False)

+-------------------------------------------------------------------+-----+
|song                                                               |count|
+-------------------------------------------------------------------+-----+
|You're The One                                                     |49   |
|Undo                                                               |34   |
|Revelry                                                            |28   |
|Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)|27   |
|Ain't Misbehavin                                                   |23   |
|Secrets                                                            |20   |
|Invalid                                                            |20   |
|Dog Days Are Over (Radio Edit)                                     |19   |
|Tive Sim                                                           |18   |
|Use Somebody                                                       |16   |
|Marry Me   

In [20]:
# w_session = Window.partitionBy('sessionId').orderBy('ts')

# df.filter('page = "Cancellation Confirmation"').select('ts', 'sessionId', 'itemInSession', 'userId', 'last_page_on_session', 'page').show() 

In [26]:
df.select(['ts', 'page', 'sessionId', 'itemInSession', 'song', 'artist']).filter('userId = 92').orderBy('ts', ascending = False).show(100, False)

+-------------+---------------+---------+-------------+-----------------------------------------------------------------------------+------------------------------+
|ts           |page           |sessionId|itemInSession|song                                                                         |artist                        |
+-------------+---------------+---------+-------------+-----------------------------------------------------------------------------+------------------------------+
|1543615837000|Add Friend     |4790     |86           |null                                                                         |null                          |
|1543615836000|Home           |4790     |85           |null                                                                         |null                          |
|1543615832000|NextSong       |4790     |84           |Fireflies                                                                    |Charttraxx Karaoke            |
|154361559

In [33]:
df_ts = df.filter('userId = 1333174').select([smin('ts').alias('min_ts'), smax('ts').alias('max_ts')]).rdd.flatMap(lambda x: x).collect()

In [51]:
def to_datetime(milliseconds, dt_format = '%Y-%m-%d %H:%M:%S'):
    return datetime.fromtimestamp(milliseconds / 1000).strftime(dt_format)  

# list(map(to_datetime, df_ts))

In [25]:
list(map(to_datetime, df.select([smin('ts').alias('min_ts'), smax('ts').alias('max_ts')]).rdd.flatMap(lambda x: x).collect()))

['2018-09-30 21:00:11', '2018-11-30 22:01:06']

In [36]:
df.groupBy('location').count().orderBy('count', ascending = False).show(50, False)

+--------------------------------------------+-----+
|location                                    |count|
+--------------------------------------------+-----+
|New York-Newark-Jersey City, NY-NJ-PA       |40156|
|Los Angeles-Long Beach-Anaheim, CA          |34278|
|Boston-Cambridge-Newton, MA-NH              |17574|
|null                                        |15700|
|Chicago-Naperville-Elgin, IL-IN-WI          |15194|
|San Francisco-Oakland-Hayward, CA           |11428|
|Atlanta-Sandy Springs-Roswell, GA           |11211|
|Phoenix-Mesa-Scottsdale, AZ                 |11184|
|Dallas-Fort Worth-Arlington, TX             |11061|
|Denver-Aurora-Lakewood, CO                  |9808 |
|Houston-The Woodlands-Sugar Land, TX        |8707 |
|Tampa-St. Petersburg-Clearwater, FL         |8330 |
|Miami-Fort Lauderdale-West Palm Beach, FL   |8180 |
|Indianapolis-Carmel-Anderson, IN            |7691 |
|Minneapolis-St. Paul-Bloomington, MN-WI     |7462 |
|Louisville/Jefferson County, KY-IN          |

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [27]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [28]:
df.filter('page = "Cancellation Confirmation"').show(50)

+------+---------+---------+------+-------------+----------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|  lastName|length|level|            location|method|                page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+----------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Cancelled|   Olivia|     F|           40|      Carr|  null| free|      Fort Wayne, IN|   GET|Cancellation Conf...|1536758439000|      490|null|   200|1538400616000|Mozilla/5.0 (Wind...|   208|
|  null|Cancelled|  Lillian|     F|          234|   Cameron|  null| paid|        Columbus, OH|   GET|Cancellation Conf...|1533472700000|      471|null|   200|1538482793000|Mozilla/5.0 (Wind...|   

In [ ]:
# Testing samples
user_id = 100010
# user_id = 121

In [147]:
CHURN_CANCELLATION_PAGE = 'Cancellation Confirmation'
REGISTRATION_PAGE = 'Submit Registration'
milliseconds_to_hours = 3600 * 1000
minutes_to_hours = 60 * 60
TRUE = 1
FALSE = 0

def transform_records(df):
    
    ts_events = df.select([smin('ts').alias('min_ts'), smax('ts').alias('max_ts')]).collect()[0]
    min_ts = ts_events[0]
    max_ts = ts_events[1]

    w_session = Window.partitionBy('sessionId').orderBy('ts')
    w_user_session = Window.partitionBy('sessionId', 'userId').orderBy('ts').rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    w_user = Window.partitionBy('userId').orderBy('ts').rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    
    # Create features
    df = df.withColumn('previous_page', lag(df.page).over(w_session))
    df = df.withColumn('last_event_ts', last(col('ts')).over(w_user))
    df = df.withColumn('last_page', last(col('page')).over(w_user))
    df = df.withColumn('cancellation_ts', when(df.last_page == CHURN_CANCELLATION_PAGE, df.last_event_ts).otherwise(max_ts))
    df = df.withColumn('register_page', first(col('previous_page')).over(w_user))
    df = df.withColumn('first_ts', first(col('ts')).over(w_user))
    df = df.withColumn('registration_ts', when(df.register_page == REGISTRATION_PAGE, df.first_ts).otherwise(min_ts))
    df = df.withColumn('ts_elapsed', last(df.ts).over(w_session) - first(df.ts).over(w_user_session))
    df = df.withColumn('session_duration', smax(df.ts_elapsed).over(w_user_session))
    
    return df
    
def create_session_dimension(df):
    
    # sessions from the user
    df_sessions = df.orderBy(df.sessionId).groupBy('sessionId', 'userId').agg(
        smax(df.ts).alias('max_event_ts'),
        smin(df.ts).alias('min_event_ts'),
        ssum(df.length).alias('session_n_total_playback'), # Based on songs length
        count(when(df.page == 'Thumbs Up', True)).alias("session_n_likes"),
        count(when(df.page == 'Thumbs Down', True)).alias("session_n_dislikes"),
        count(when(df.page == 'NextSong', True)).alias("session_n_songs"),
        count(when(df.page == 'Add Friend', True)).alias("session_n_friends"),
        count(when(df.page == 'Add to Playlist', True)).alias("session_n_add_playlist"),
        count(when(df.page == 'Home', True)).alias("session_n_home"),
        count(when(df.page == 'Roll Advert', True)).alias("session_n_ads"),
        count(when(df.page == 'Help', True)).alias("session_n_help"),
        count(when(df.page == 'Error', True)).alias("session_n_error"),
        count(when(df.page == 'Settings', True)).alias("session_n_sets"),
        count(col('page')).alias('session_n_actions'),
        first(col('session_duration')).alias('session_duration')
    ) 

    w_user_sessions_interval = Window.partitionBy('userId').orderBy('min_event_ts')
    
    # Calculate the interval until the next session
    df_sessions = df_sessions.withColumn('interval_to_session', col('min_event_ts') - lag(col('max_event_ts')).over(w_user_sessions_interval))
    
    # Calculate average time in hours for each session
    df_session_time = df_sessions.groupBy('userId').agg(
       (avg(df_sessions.session_duration) / milliseconds_to_hours).alias('session_hours')#,
       #(ssum(df_sessions.session_duration) / milliseconds_to_hours).alias('total_hours')# could be sum or avg?
    )
    df_sessions = df_sessions.join(df_session_time, on = 'userId')
    
    # We should remove the null lines before count/group to not account 2 times the mean interval
    df_sessions = df_sessions.groupBy('userId').agg( 
        #first(col('total_hours')).alias('total_hours'),
        (avg(df_sessions.interval_to_session) / milliseconds_to_hours).alias('session_avg_time_away'),
        ((avg(df_sessions.session_n_total_playback) / minutes_to_hours) / first(col('session_hours'))).alias('session_avg_playback'), 
        (avg(df_sessions.session_n_likes) / first(col('session_hours'))).alias('session_avg_likes'),
        (avg(df_sessions.session_n_dislikes) / first(col('session_hours'))).alias('session_avg_dislikes'),
        (avg(df_sessions.session_n_songs) / first(col('session_hours'))).alias('session_avg_songs'),
        (avg(df_sessions.session_n_friends) / first(col('session_hours'))).alias('session_avg_friends'),
        (avg(df_sessions.session_n_add_playlist) / first(col('session_hours'))).alias('session_avg_added_playlist'),
        (avg(df_sessions.session_n_home) / first(col('session_hours'))).alias('session_avg_home'),
        (avg(df_sessions.session_n_ads) / first(col('session_hours'))).alias('session_avg_ads'),
        (avg(df_sessions.session_n_help) / first(col('session_hours'))).alias('session_avg_help'),
        (avg(df_sessions.session_n_error) / first(col('session_hours'))).alias('session_avg_errors'),
        (avg(df_sessions.session_n_sets) / first(col('session_hours'))).alias('session_avg_settings'),
        (avg(df_sessions.session_n_actions) / first(col('session_hours'))).alias('session_avg_actions')
    )
    
    return df_sessions

def create_user_dimension(df):
    
    df_user_profile = df.groupby('userId')\
        .agg( 

            first(col('state')).alias('state'),
            first(when(col('gender') == 'M', TRUE).otherwise(FALSE)).alias('male'),

            smin(col('first_ts')).alias('ts_start'),
            smax(col('last_event_ts')).alias('ts_end'),        
        
            ((smax(col('last_event_ts')) - smin(col('first_ts'))) / milliseconds_to_hours).alias('time_window'),
        
            # Subscription
            count(when(col('page') == 'Submit Downgrade', True)).alias('n_downgrades'),
            count(when(col('page') == 'Submit Upgrade', True)).alias('n_upgrades'),
            last(when(col('level') == 'paid', TRUE).otherwise(FALSE)).alias('paid'),
            first(when(col('last_page') == CHURN_CANCELLATION_PAGE, TRUE).otherwise(FALSE)).alias('canceled'),

            # Streaming
            count(when(col('page') == 'NextSong', True)).alias('n_songs'),
            count(when(col('page') == 'Thumbs Up', True)).alias('n_likes'),
            count(when(col('page') == 'Thumbs Down', True)).alias('n_dislikes'),
            countDistinct(col('sessionId')).alias('n_sess'),
            (avg(col('session_duration')) / milliseconds_to_hours).alias('avg_session_duration'),

            # Community
            count(when(col('page') == 'Add Friend', True)).alias('n_friends'),
            count(when(col('page') == 'Add to Playlist', True)).alias('n_added_to_playlist'),

            # Other
            count(when(col('page') == 'Home', True)).alias('n_home'),
            count(when(col('page') == 'Roll Advert', True)).alias('n_ads'),
            count(when(col('page') == 'Help', True)).alias('n_help'),
            count(when(col('page') == 'Error', True)).alias('n_errors'),
            count(when(col('page') == 'Settings', True)).alias('n_settings'),
            count(col('page')).alias('n_actions')
        )
    
    
    # Location
    states = list(map(lambda c: c[0].strip(), df.select(['state']).distinct().rdd.collect()))
    
    for state in states:
        df_user_profile = df_user_profile.withColumn(state.lower(), when(df_user_profile.state == state, 1).otherwise(0))
    
    return df_user_profile

def create_days_dimension(df):
    
    df_unique_days = df.groupby('userId').agg(countDistinct('date').alias('n_days'))
    
    df_daily_actions = df.groupby('userId', 'date').agg(count('page').alias('total'))
    df_daily_actions = df_daily_actions.groupby('userId').agg(avg('total').alias('avg_daily_actions')) 

    df_days = df_unique_days.join(df_daily_actions, df_unique_days.userId == df_daily_actions.userId)
    
    # Remove duplicated column
    df_days = df_days.drop(df_daily_actions.userId)
    
    return df_days

In [82]:
transform_records(df).select(col('session_duration'), col('sessionId'), col('userId')).where(col('userId') == 100010).groupBy('userId').agg(ssum(col('session_duration'))).show(5)

+------+---------------------+
|userId|sum(session_duration)|
+------+---------------------+
|100010|           1730014000|
+------+---------------------+



In [135]:
df = transform_records(df)

df_sessions = create_session_dimension(df)

df_sessions.show(2, vertical = True)

-RECORD 0-------------------------------------------
 userId                      | 100010               
 session_avg_time_away       | 66.0225              
 first(session_hours, false) | 3.50625              
 session_avg_playback        | 1.0247950382253912   
 session_avg_likes           | 0.5704099821746881   
 session_avg_dislikes        | 0.42780748663101603  
 session_avg_songs           | 13.689839572192513   
 session_avg_friends         | 0.42780748663101603  
 session_avg_added_playlist  | 0.14260249554367202  
 session_avg_home            | 0.42780748663101603  
 session_avg_ads             | 3.1372549019607843   
 session_avg_help            | 0.0                  
 session_avg_errors          | 0.0                  
 session_avg_settings        | 0.0                  
 session_avg_actions         | 19.536541889483065   
-RECORD 1-------------------------------------------
 userId                      | 200002               
 session_avg_time_away       | 166.99152777777

In [134]:
def sort_features(df, columns_order):
    _columns = df.columns
    _columns.sort()
    
    for _idx, _val in list(enumerate(columns_order)):
        _columns.pop(_columns.index(_val))
        _columns.insert(_idx, _val)
        
    assert len(_columns) == len(df.columns)

    return _columns

In [148]:
df = transform_records(df)

df_sessions = create_session_dimension(df)
df_days = create_days_dimension(df)

df_users = create_user_dimension(df)

df_users = df_users.orderBy(df_users.userId).join(df_days, on = 'userId')

_columns = list(set(df_users.schema.names + df_sessions.schema.names) - set(['ts_start', 'ts_end']))

df_users = df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns) 

# Enforces the order for some columns
_columns = sort_features(df_users, [ 'userId', 'male', 'paid', 'canceled'])

### WARN: Only round to display
df_users.select([sround(c, 0).cast(dataType = IntegerType()).alias(c) for c in _columns]).fillna(0).show(2, True, vertical = True)

-RECORD 0----------------------------
 userId                     | 100010 
 male                       | 0      
 paid                       | 0      
 canceled                   | 1      
 ak                         | 0      
 al                         | 0      
 ar                         | 0      
 avg_daily_actions          | 69     
 avg_session_duration       | 4      
 az                         | 0      
 ca                         | 0      
 co                         | 0      
 ct                         | 1      
 dc                         | 0      
 fl                         | 0      
 ga                         | 0      
 hi                         | 0      
 ia                         | 0      
 id                         | 0      
 il                         | 0      
 in                         | 0      
 ks                         | 0      
 ky                         | 0      
 la                         | 0      
 ma                         | 0      
 md         

In [149]:
len(df_users.columns)

85

In [150]:
df_users.select(_columns).fillna(0).toPandas().to_csv('sparkify_data_final.csv', index = False)

In [549]:
df.agg(countDistinct(df.userId).alias('unique_users')).show()

+------------+
|unique_users|
+------------+
|         449|
+------------+



In [544]:
df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns).count()

425

In [548]:
df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns).groupBy('canceled').agg(count(df_users.canceled).alias('total')).show()

+--------+-----+
|canceled|total|
+--------+-----+
|       1|   90|
|       0|  335|
+--------+-----+



- Advertises number (per session and all)
    - The user **100010** returned after some idle time and received a considerable amount of advertises;
    - Also, after thumbs down, I received two advertisements on four sounds. Then canceled the service.
- Number of sessions
- Paid subscription time
- Avg songs before an ad
- Number of skipped songs

In [553]:
df.schema.names

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [ ]:
to_date(df.ts.cast(dataType=TimestampType()))

In [581]:
df.where(df.userId == user_id).select(['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level', 
 'page',
 'sessionId',
 'song', 
 'ts', 
 'userId']).orderBy('sessionId', 'itemInSession').withColumn('datetime', date_format((df.ts/1000).cast(dataType=TimestampType()), 'HH:mm:ss dd-MM-YYYY')).show(350, True)

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+---------+--------------------+-------------+------+-------------------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|                page|sessionId|                song|           ts|userId|           datetime|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+---------+--------------------+-------------+------+-------------------+
|              Darude|Logged In| Darianna|     F|            0|Carpenter|226.08934| free|            NextSong|       62|           Sandstorm|1538991392000|100010|06:36:32 08-10-2018|
|             Justice|Logged In| Darianna|     F|            1|Carpenter|285.41342| free|            NextSong|       62|Phantom Part 1.5 ...|1538991618000|100010|06:40:18 08-10-2018|
|    Five Iron Frenzy|Logged In| Darianna|     F|            2|Carpenter|236.09424| f

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler

In [18]:
columns_to_exclude = set(['userId'])

columns_to_use = list(set(df_users.columns) - columns_to_exclude)

columns_to_train = list(set(columns_to_use) - set(['canc']))

columns_to_use.sort()
columns_to_train.sort()

print(f'Columns: {columns_to_use}\n')
print(f'Columns to train: {columns_to_train}')

Columns: ['a_acti', 'a_adde', 'a_ads', 'a_disl', 'a_erro', 'a_frie', 'a_help', 'a_home', 'a_like', 'a_play', 'a_sett', 'a_song', 'a_stim', 'a_tiaw', 'canc', 'male', 'n_acti', 'n_adde', 'n_ads', 'n_disl', 'n_down', 'n_erro', 'n_frie', 'n_help', 'n_home', 'n_like', 'n_sess', 'n_sett', 'n_song', 'n_upgr', 'paid']

Columns to train: ['a_acti', 'a_adde', 'a_ads', 'a_disl', 'a_erro', 'a_frie', 'a_help', 'a_home', 'a_like', 'a_play', 'a_sett', 'a_song', 'a_stim', 'a_tiaw', 'male', 'n_acti', 'n_adde', 'n_ads', 'n_disl', 'n_down', 'n_erro', 'n_frie', 'n_help', 'n_home', 'n_like', 'n_sess', 'n_sett', 'n_song', 'n_upgr', 'paid']


In [24]:
CHURN_LABEL = 'canc'
TRAIN_SPLIT_RATIO = .8
TEST_SPLIT_RATIO = .2

SPLIT_RATIO = [TRAIN_SPLIT_RATIO, TEST_SPLIT_RATIO]

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def evaluate_multiclass_classifier(predictions, columns):
    metrics_to_evaluate = [ 'accuracy', 'f1', 'weightedPrecision', 'weightedRecall' ]
    
    result = {}
    for metric in metrics_to_evaluate:
        evaluator = MulticlassClassificationEvaluator(labelCol = columns[0], predictionCol = columns[1], metricName = metric)
        value = evaluator.evaluate(predictions)
        result[metric] = value
        print(f'{metric}: {value}') 
    
    return result

def train_random_forest_classifier(df, columns, train_cloumns):
    
    # Create the new dataframe
    data = df.select(columns).fillna(0)
    
    # Split train/test
    (train_df, test_df) = data.randomSplit(SPLIT_RATIO, seed = 42)
    
    # Create the indexer for labels
    l_indexer = StringIndexer(inputCol = CHURN_LABEL, outputCol = 'idx_labels')
    
    # Create the feature tranformer, to generate an array representation of all features on dataset
    f_assembler = VectorAssembler(inputCols = train_cloumns, outputCol = 'features')
    
    # Create the model instance
    rf_classifier = RandomForestClassifier(labelCol = 'idx_labels', featuresCol = 'features', numTrees=10)

    # Converts the predictions to original labels
    l_translator = IndexToString(inputCol = 'prediction', outputCol = 'predictedLabel', labels = [ 'Not churn', 'Churn' ])

    # Create the pipeline
    pipeline = Pipeline(stages = [ l_indexer, f_assembler, rf_classifier, l_translator ])

    # Train the model
    model = pipeline.fit(train_df)

    # Test the model
    predictions = model.transform(test_df)

    return model.stages[2], predictions
    
from pyspark.ml.feature import StandardScaler

binary_features = [ 'paid', 'male' ]
numeric_features = ['a_acti', 'a_adde', 'a_ads', 'a_play', 'a_sett', 'a_song', 'a_stim', 'a_tiaw', 
                    'male', 'n_acti', 'n_adde', 'n_ads', 'n_disl', 'n_down', 'n_erro', 'n_frie', 'n_help', 'n_home',
                    'n_like', 'n_sess', 'n_sett', 'n_song', 'n_upgr' ]
    

def create_pipeline(model):
    l_indexer = StringIndexer(inputCol = CHURN_LABEL, outputCol = 'idx_labels')
    f_binaries = VectorAssembler(inputCols = binary_features, outputCol = 'bin_features')
    f_numeric = VectorAssembler(inputCols = numeric_features, outputCol = 'num_features')
    f_scaler = StandardScaler(inputCol = 'num_features', outputCol = 'num_features_escaled', withStd = True, withMean = True)
    f_all = VectorAssembler(inputCols = [ 'bin_features' , 'num_features_escaled' ], outputCol = 'features')
    pipeline = Pipeline(stages = [ l_indexer, f_binaries, f_numeric, f_scaler, f_all, model ])
    return pipeline

def create_random_forest_pipeline():
    rf_classifier = RandomForestClassifier(labelCol = 'canc', featuresCol = 'features', seed = 42)
    return create_pipeline(rf_classifier)

def create_gradient_boost_pipeline():
    gbt_classifier = GBTClassifier(labelCol = 'canc', maxDepth = 5, maxIter = 100)
    return create_pipeline(gbt_classifier)

In [145]:
model, predictions = train_random_forest_classifier(df_users, columns_to_use, columns_to_train)

In [150]:
evaluate_multiclass_classifier(predictions, ('canc', 'prediction'))

accuracy: 0.8181818181818182
f1: 0.7823601642884364
weightedPrecision: 0.8007451564828614
weightedRecall: 0.8181818181818181


In [151]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')

evaluator.evaluate(predictions)

0.7091346153846155

In [31]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def create_grid_search(pipeline, param_grid, evaluator = BinaryClassificationEvaluator(labelCol = 'canc')):
    
    return CrossValidator(estimator = pipeline, estimatorParamMaps = param_grid, evaluator = evaluator, numFolds = 3, parallelism = 10)

def random_forest_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    grid_rf = ParamGridBuilder().addGrid(model.maxDepth, [5]) #[5, 10, 15, 20, 25]) 
    grid_rf = grid_rf.addGrid(model.numTrees, [40]) #[20, 40, 60, 70])
    grid_rf = grid_rf.build()
        
    return create_grid_search(pipeline, grid_rf)

def gradient_boost_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    grid_gbt = ParamGridBuilder().addGrid(model.maxDepth, [5]) #, 10, 15, 20, 25])
    grid_gbt = grid_gbt.addGrid(model.maxIter, [20])#, 25, 40, 50, 100])
    grid_gbt = grid_gbt.build()
   
    return create_grid_search(pipeline, grid_gbt)

In [28]:
# Create the new dataframe
data = df_users.select(columns_to_use).fillna(0)

# Split train/test
(train_df, test_df) = data.randomSplit(SPLIT_RATIO, seed = 42)

In [23]:
train_df.cache()
test_df.cache()

DataFrame[a_acti: double, a_adde: double, a_ads: double, a_disl: double, a_erro: double, a_frie: double, a_help: double, a_home: double, a_like: double, a_play: double, a_sett: double, a_song: double, a_stim: double, a_tiaw: double, canc: int, male: int, n_acti: bigint, n_adde: bigint, n_ads: bigint, n_disl: bigint, n_down: bigint, n_erro: bigint, n_frie: bigint, n_help: bigint, n_home: bigint, n_like: bigint, n_sess: bigint, n_sett: bigint, n_song: bigint, n_upgr: bigint, paid: int]

In [32]:
pipeline = create_random_forest_pipeline()
cv_rf = random_forest_grid_search(pipeline)

In [33]:
cv_rf_results = cv_rf.fit(train_df)

KeyboardInterrupt: 

In [24]:
pipeline = create_gradient_boost_pipeline()
cv_gbt = gradient_boost_grid_search(pipeline)

In [25]:
cv_gbt_results = cv_gbt.fit(train_df)

KeyboardInterrupt: 

In [65]:
cv_gbt_results.bestModel

PipelineModel_441706ed8036

In [ ]:
import pandas as pd

scores = cv_gbt_results.avgMetrics
params = [{p.name: v for p, v in m.items()} for m in cv_gbt.getEstimatorParamMaps()]
params_pd = pd.DataFrame(params)
params_pd['score'] = scores
params_pd

In [24]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')
 
best_model_results = cv_gbt_results.bestModel.transform(test_df)
    
evaluator.evaluate(best_model_results)

0.7385507246376812

In [251]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')
 
best_model_results = cv_rf_results.bestModel.transform(test_df)
    
evaluator.evaluate(best_model_results)

0.7489855072463769

In [25]:
evaluate_multiclass_classifier(best_model_results, ('canc', 'prediction'))

accuracy: 0.7653061224489796
f1: 0.7547809052810739
weightedPrecision: 0.7487244897959183
weightedRecall: 0.7653061224489796


{'accuracy': 0.7653061224489796,
 'f1': 0.7547809052810739,
 'weightedPrecision': 0.7487244897959183,
 'weightedRecall': 0.7653061224489796}

In [252]:
evaluate_multiclass_classifier(best_model_results, ('canc', 'prediction'))

accuracy: 0.7959183673469388
f1: 0.7674625405717843
weightedPrecision: 0.7739108182457936
weightedRecall: 0.7959183673469388


{'accuracy': 0.7959183673469388,
 'f1': 0.7674625405717843,
 'weightedPrecision': 0.7739108182457936,
 'weightedRecall': 0.7959183673469388}

In [237]:
best_model_results.select(['features', 'prediction', 'canc']).show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |prediction|canc|
+-------------

In [204]:
best_model_results.select(['rawPrediction', 'prediction', 'canc']).show(10, False)

+--------------------+----------+----+
|       rawPrediction|prediction|canc|
+--------------------+----------+----+
|[7.36601764248823...|       1.0|   1|
|[17.0575710379575...|       0.0|   1|
|[16.4893218608728...|       0.0|   1|
|[16.3220158268161...|       0.0|   1|
|[10.4271757825937...|       0.0|   0|
|[13.8699023989112...|       0.0|   1|
|[14.3856274927273...|       0.0|   0|
|[16.2067887268884...|       0.0|   0|
|[15.0503712183422...|       0.0|   1|
|[18.0768931186343...|       0.0|   0|
+--------------------+----------+----+
only showing top 10 rows



In [130]:
test_df.filter('canc = 1').count()

24

In [131]:
train_df.filter('canc = 1').count()

75

In [26]:
best_model_results.select("prediction", "canc", "features").filter('canc = 1').groupby(['canc', 'prediction']).agg({'canc':'count'}).show(50)

+----+----------+-----------+
|canc|prediction|count(canc)|
+----+----------+-----------+
|   1|       0.0|         14|
|   1|       1.0|          9|
+----+----------+-----------+



In [269]:
cv_rf_results.bestModel.stages[-1]

RandomForestClassificationModel (uid=RandomForestClassifier_18ab5d75a053) with 20 trees

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.